# Understanding Pandas Series and DataFrames - Comprehensive Lab

## Introduction
This notebook provides a hands-on exploration of pandas Series and DataFrames, focusing on:
- Data cleaning and transformation
- Using `.map()` and `.apply()` methods
- Manipulating DataFrame structure
- Working with indices and data types
- Grouping and analysis techniques

Let's dive in and build practical data skills!

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Set some display options for better visibility
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)

# Increase the number of rows pandas will display
#pd.set_option('display.max_rows', None)

# Optionally, reset back to default later if needed
# pd.reset_option('display.max_rows')

# 1. Reading and Exploring Data

In [ ]:
#  reading in the turnstile data
df = pd.read_csv('turnstile_180901.txt')

print(f"DataFrame Shape: {df.shape} (rows, columns)")
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst 5 rows:")
df.head()

In [ ]:
# Basic Data Inspection
# Looking at data types and checking for missing values
print("DataFrame info (data types and non-null counts):")
df.info()

print("\nSummary statistics for numeric columns:")
df.describe()

# 2. Cleaning Column Names - Using List Comprehension

In [ ]:
# Original column names
print("Original column names:")
print(df.columns)

# Understanding list comprehension for renaming columns:
# - 'col' is a temporary variable that takes each value from df.columns
# - .lower() converts each column name to lowercase
# - We create a new list of lowercase names and assign it back to df.columns
df.columns = [col.lower() for col in df.columns]

print("\nLowercase column names:")
print(df.columns)

# Advanced renaming using a custom function
def clean_column_name(name):
    """Function to standardize column names"""
    # Convert to lowercase, replace spaces with underscores
    cleaned = name.lower().replace(' ', '_').replace('/', '_')
    return cleaned

# We could also use this approach:
#df.columns = [clean_column_name(col) for col in df.columns]

In [ ]:
# First, let's check what the column names actually look like and if they have whitespace
print("Column names with their string representation (to see any whitespace):")
for col in df.columns:
    print(f"'{col}' (length: {len(col)})")

# Strip whitespace from all column names
df.columns = df.columns.str.strip()
print("\nColumn names after stripping whitespace:")
print(df.columns.tolist()) #.tolist() converts the Index object to a list for better readability


In [ ]:
# .map() - Used for mapping values in a Series based on a dictionary or function
# Let's create a day of week mapping first by converting dates to datetime
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.dayofweek  # 0=Monday, 6=Sunday

# Map day numbers to day names
day_mapping = {
    0: 'Monday', 
    1: 'Tuesday', 
    2: 'Wednesday', 
    3: 'Thursday', 
    4: 'Friday', 
    5: 'Saturday', 
    6: 'Sunday'
}

# Using map to convert day numbers to day names
df['day_name'] = df['day_of_week'].map(day_mapping)

# Create a weekend indicator using map
weekend_map = {0: False, 1: False, 2: False, 3: False, 4: False, 5: True, 6: True}
df['is_weekend'] = df['day_of_week'].map(weekend_map)

# View the results
df[['date', 'day_of_week', 'day_name', 'is_weekend']].head(10)

# 3. Introduction to .map() and .apply()

In [ ]:
# .apply() - More versatile than map, works with custom functions
# Example 1: Simple apply to a Series
# Count the number of characters in the station name
df['station_name_length'] = df['station'].apply(len)

# Example 2: Apply a more complex function 
def count_train_lines(line_name):
    """Counts the number of train lines that pass through a station"""
    # Each character typically represents a different train line
    # Remove non-alphabetic characters for better counting
    clean_name = ''.join(c for c in line_name if c.isalpha())
    return len(clean_name)

# Apply our function to the linename column
df['num_lines'] = df['linename'].apply(count_train_lines)

# Preview the results
df[['station', 'station_name_length', 'linename', 'num_lines']].head()

# 4. Working with DataFrame Indices


In [ ]:
# Default index
print("Current index:")
print(df.index)

# Setting a column as the index
df_indexed = df.set_index('linename')
print("\nAfter setting 'linename' as the index:")
print(df_indexed.index)
df_indexed.head()

In [ ]:
# Resetting the index (turning the index back into a regular column)
df_reset = df_indexed.reset_index()
print("After resetting the index:")
print(df_reset.index)
df_reset.head()



## Read on your own on Multi-level Indexing 
### What is Multi-level Indexing?
Multi-level (or hierarchical) indexing allows you to have multiple levels of indices in your DataFrame. Think of it like organizing data in nested categories:

- First level: Major category (in this case, station)
- Second level: Sub-category (in this case, linename)

In [ ]:
# Create a multi-level index and sort it for proper performance
df_multi = df.set_index(['station', 'linename']).sort_index()

print("Multi-level index structure:")
print(df_multi.index.names)  # Shows the names of each level

# Display the first few entries
print("\nFirst few entries with multi-level index:")
df_multi.head()

# Check if the specific combination exists
stations = df_multi.index.get_level_values('station').unique()
if '59 ST' in stations:
    print("\n'59 ST' exists as a station")
    
    # Get all the line names for 59 ST
    try:
        lines_at_59st = df_multi.loc['59 ST'].index.unique()
        print(f"Line names at 59 ST: {lines_at_59st.tolist()}")
        
        # If NQR456W exists, show its data
        if 'NQR456W' in lines_at_59st:
            print("\nData for 59 ST, NQR456W:")
            print(df_multi.loc[('59 ST', 'NQR456W')].head())
    except:
        print("Error accessing station data")
else:
    print("'59 ST' station not found")

# 5. Changing Data Types

## Difference Between `int64` and `int32` in Pandas & NumPy  
Both `int64` and `int32` are integer data types, but they differ in **memory usage** and **value range**.

| **Data Type** | **Storage Size** | **Value Range** |
|--------------|----------------|----------------------------|
| `int32` | **4 bytes (32 bits)** | -2,147,483,648 to 2,147,483,647 |
| `int64` | **8 bytes (64 bits)** | -9,223,372,036,854,775,808 to 9,223,372,036,854,775,807 |

---

### **Key Differences**
1. **Memory Usage**  
   - `int32` **uses 4 bytes** per value → better for memory efficiency.  
   - `int64` **uses 8 bytes** per value → needed for very large numbers.  

2. **Number Range**  
   - `int32` can store values up to **±2.1 billion**.  
   - `int64` can store **much larger values**, up to ±9 quintillion.  

3. **Performance Considerations**  
   - `int32` operations **use less memory**, making computations slightly faster.  
   - `int64` is required for **big data processing** (e.g., large IDs or financial data).  

---

In [ ]:
# Check current data types
print("Current dtypes:")
print(df.info())

# Convert numeric columns to more efficient types
# Let's focus on 'entries' and 'exits'
print("\nMemory usage before optimization:")
print(df[['entries', 'exits']].memory_usage())

# Convert to smaller integer types where appropriate
df['entries'] = df['entries'].astype('int32')
df['exits'] = df['exits'].astype('int32')

print("\nMemory usage after optimization:")
print(df[['entries', 'exits']].memory_usage())

# Working with dates
# We already converted 'date' to datetime
print("\nDate range in the dataset:")
print(f"Start date: {df['date'].min()}")
print(f"End date: {df['date'].max()}")

# 6. Grouping and Aggregating Data

In [ ]:
# Group by day of week and calculate average entries/exits
# when selecting multiple columns, use a list (double brackets) not a tuple
day_grouped = df.groupby('day_name')[['entries', 'exits']].mean()
day_grouped = day_grouped.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
print("Average entries/exits by day of week:")
day_grouped



In [ ]:
# Group by station and count records
station_counts = df.groupby('station').size().sort_values(ascending=False)
print("\nTop stations by number of records:")
station_counts.head(10)



In [ ]:
# Multiple aggregations with groupby
station_stats = df.groupby('station').agg({
    'entries': ['min', 'max', 'mean'],
    'exits': ['min', 'max', 'mean'],
    'linename': 'nunique'  # Count unique line names
})
print("\nMultiple statistics by station:")
station_stats.head()

# 7. Sorting DataFrames

In [ ]:
# Sort by entries (highest first)
df_sorted = df.sort_values('entries', ascending=False)
print("Top entries records:")
df_sorted[['station', 'date', 'time', 'entries']].head()


In [ ]:
# Sort by multiple columns
# For example, by station name and then by date
df_multi_sorted = df.sort_values(['station', 'date', 'time'])
print("\nSorted by station, date, and time:")
df_multi_sorted[['station', 'date', 'time', 'entries']].head()

# 8. Creating New Calculated Columns

In [ ]:
# Let's extract the hour from the time column
# First, let's convert time to a proper time format
df['time'] = pd.to_datetime(df['time']).dt.time
df['hour'] = pd.to_datetime(df['time'], format='%H:%M:%S').dt.hour
df[['time', 'hour']].head()


In [ ]:
# # Let's extract the hour from the time column
# # First, let's convert time to a proper time format
# df['time'] = pd.to_datetime(df['time'], format='%H:%M:%S')  # Keep full datetime
# df['hour'] = df['time'].dt.hour # Extract hour from time
# print("\nFirst few rows with extracted hour:")
# df[['time', 'hour']].head()


In [ ]:
# Create a 'time_of_day' column
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'

df['time_of_day'] = df['hour'].apply(time_of_day)

# View distribution of records by time of day
time_of_day_counts = df['time_of_day'].value_counts()
print("Records by time of day:")
time_of_day_counts

# 9. Visualizing Data

In [ ]:
# Entries by day of week
day_entries = df.groupby('day_name')['entries'].sum()
day_entries = day_entries.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])

plt.figure(figsize=(10, 6))
day_entries.plot(kind='bar', color='skyblue')
plt.title('Total Entries by Day of Week')
plt.ylabel('Total Entries')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
# Weekend vs. Weekday comparison
weekend_comparison = df.groupby('is_weekend')[['entries', 'exits']].sum()
weekend_comparison.index = ['Weekday', 'Weekend']

plt.figure(figsize=(8, 5))
weekend_comparison.plot(kind='bar', colormap='Set2')
plt.title('Entries and Exits: Weekend vs. Weekday')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.legend(['Entries', 'Exits'])
plt.tight_layout()
plt.show()



In [ ]:
# Time of day analysis
time_entries = df.groupby('time_of_day')['entries'].sum()
time_entries = time_entries.reindex(['Morning', 'Afternoon', 'Evening', 'Night'])

plt.figure(figsize=(8, 5))
time_entries.plot(kind='pie', autopct='%1.1f%%', startangle=90, colors=['lightblue', 'lightgreen', 'coral', 'lavender'])
plt.title('Distribution of Entries by Time of Day')
plt.ylabel('')
plt.tight_layout()
plt.show()

# 10. Handling Missing Data and Data Cleaning


In [ ]:
pd.reset_option('display.max_rows') #

# Check for missing values
print("Missing values in each column:")
df.isnull().sum()


## If we did have missing values, here's how we could handle them:

### Example 1: Fill missing values with a specific value
``` df['entries'] = df['entries'].fillna(0) ```

### Example 2: Fill missing values with the column mean
``` df['entries'] = df['entries'].fillna(df['entries'].mean()) ```

### Example 3: Forward fill (use the previous value)
``` df = df.fillna(method='ffill') ```

### Example 4: Drop rows with any missing values
``` df_clean = df.dropna() ```

In [ ]:
# Example 5: Detect and handle outliers
# Using IQR method to identify outliers
Q1 = df['entries'].quantile(0.25)
Q3 = df['entries'].quantile(0.75)
IQR = Q3 - Q1

# Define outliers as values outside 1.5 * IQR from Q1 or Q3
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print("\nOutlier boundaries for 'entries':")
print(f"Lower bound: {lower_bound}")
print(f"Upper bound: {upper_bound}")

# Count outliers
outliers = df[(df['entries'] < lower_bound) | (df['entries'] > upper_bound)]
print(f"\nNumber of outlier records: {len(outliers)}")

# 11. Removing Columns


In [ ]:
# Let's say we want to remove unnecessary columns
columns_to_drop = ['c/a', 'scp']
df_slim = df.drop(columns=columns_to_drop)

print(f"Original columns: {df.columns.tolist()}")
print(f"Columns after dropping: {df_slim.columns.tolist()}")

# Alternative methods:
# df_slim = df.drop(['c/a', 'scp'], axis=1)  # Same result
# To modify in place:
# df.drop(columns=columns_to_drop, inplace=True)

What Does inplace=True Mean in Pandas?
In Pandas, inplace=True is an argument used in various DataFrame methods to modify the data directly, without creating a new copy.

How inplace=True Works
- When inplace=False (default), the operation returns a new modified DataFrame, leaving the original unchanged.
- When inplace=True, the operation modifies the DataFrame in place, meaning no new DataFrame is returned.

In [ ]:
# import pandas as pd

# # Sample DataFrame
# df = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})

# # Without inplace=True (Creates a new DataFrame)
# df_new = df.drop(columns=['B'])
# print()
# print(df_new)  # 'B' is removed
# print(df)      # Original df is unchanged

# # # With inplace=True (Modifies df directly)
# # df.drop(columns=['B'], inplace=True)
# # print(df)  # 'B' is permanently removed

# 12. Analysis Summary and Potential Misleading Visualizations

In [ ]:
# Summary of what we've learned about the dataset
print("SUMMARY OF ANALYSIS:")
print(f"1. Dataset spans from {df['date'].min()} to {df['date'].max()}")
print(f"2. Contains data from {df['station'].nunique()} unique stations")
print(f"3. Most common station: {df['station'].value_counts().idxmax()}")
print(f"4. Weekend vs Weekday entries ratio: {weekend_comparison.loc['Weekend', 'entries'] / weekend_comparison.loc['Weekday', 'entries']:.2f}")

# Discussion of potentially misleading aspects in the visualizations
print("\nPOTENTIAL MISLEADING ASPECTS IN THE ANALYSIS:")
print("1. Day of week charts could be misleading because:")
print("   - Weekend days (2 days) are compared with weekday data (5 days)")
print("   - We're looking at totals, not averages per day")
print("   - Data might not account for holidays or special events")
print("2. Station traffic patterns might vary significantly by location")
print("3. Without normalizing for number of turnstiles, busier stations may be overrepresented")

# 13. Exporting Processed Data

In [ ]:
# Save to CSV
df_slim.to_csv('processed_turnstile_data.csv', index=False)

# Save to Excel with multiple sheets
with pd.ExcelWriter('turnstile_analysis.xlsx') as writer:
    df_slim.head(1000).to_excel(writer, sheet_name='Raw_Data', index=False)
    day_grouped.to_excel(writer, sheet_name='Day_Analysis')
    weekend_comparison.to_excel(writer, sheet_name='Weekend_Comparison')
    
print("Data exported to CSV and Excel files")

#### Save to CSV (default: index=True)
``` df.to_csv('output.csv') ```

Output (inside 'output.csv'):
,A,B
0,10,40
1,20,50
2,30,60

In this comprehensive pandas lab, we've learned:

1. Data Import and Exploration:
   - Reading CSV files and examining structure
   - Checking data types and missing values

2. Data Cleaning and Transformation:
   - Renaming columns and standardizing names
   - Converting data types for analysis
   - Creating derived fields (day of week, time of day)

3. Data Analysis Techniques:
   - Using .map() for value mapping 
   - Using .apply() for complex transformations
   - Grouping data for aggregation
   - Sorting for exploratory analysis

4. Index Manipulation:
   - Setting and resetting indices
   - Working with multi-level indices

5. Visualization:
   - Creating insightful plots
   - Understanding potential misleading aspects

The MTA turnstile dataset provides rich insights into usage patterns by day of week, time of day, and location. Such analysis can help optimize staffing, maintenance schedules, and improve passenger experience.
